In [ ]:
#把資料load進dataFrame裡面(約耗時25分鐘)

#以下是採用每100萬筆存入一個df，再將其concate，因此多少個df要視每個batch不同而定
import pandas as pd
import json
from pandas.io.json import json_normalize
from datetime import datetime

df = pd.DataFrame()
with open('D:\json_data\part_00000.json', 'r',encoding="utf-8") as f: #此處資料路徑須改成你電腦的路徑
    ls = f.readlines()
    lines = []
    start_time = datetime.now()
    for line in ls[:1000000]:
        
        lines.append(json.loads(line))
    print("+")
    df1 = pd.DataFrame.from_dict(pd.json_normalize(lines), orient='columns')
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    lines = []
    start_time = datetime.now()
    for line in ls[1000000:2000000]:
        
        lines.append(json.loads(line)) 
    print("-")
    df2 = pd.DataFrame.from_dict(pd.json_normalize(lines), orient='columns')
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    lines = []
    start_time = datetime.now()
    for line in ls[2000000:3000000]:
        
        lines.append(json.loads(line)) 
    print("-")
    df3 = pd.DataFrame.from_dict(pd.json_normalize(lines), orient='columns')
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    lines = []
    start_time = datetime.now()
    for line in ls[3000000:4000000]:
        
        lines.append(json.loads(line)) 
    print("-")
    df4 = pd.DataFrame.from_dict(pd.json_normalize(lines), orient='columns')
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    lines = []
    start_time = datetime.now()
    for line in ls[4000000:5000000]:  
        lines.append(json.loads(line)) 
    print("-")
    df5 = pd.DataFrame.from_dict(pd.json_normalize(lines), orient='columns')
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    lines = []
    start_time = datetime.now()
    for line in ls[5000000:]:
        
        lines.append(json.loads(line)) 
    print("-")
    df6 = pd.DataFrame.from_dict(pd.json_normalize(lines), orient='columns')
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    start_time = datetime.now()
    df = pd.concat([df1,df2,df3,df4,df5,df6], axis=0, ignore_index=True)
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

In [ ]:
#查看batch有幾位使用者
df.f_device_id.nunique()

In [ ]:
#查看當在使用各個功能時popup出訂閱視窗，而使用者點擊start free trial(主辦定義這樣就符合subscriber)的次數
show = df["e_segment_map.operation"]
new = show[show.values == "subscribe"]
len(new.index)

#因為次數不代表人數，此步驟找出人數
import numpy as np
opeSub = df.loc[np.array(new.index),"f_device_id"]
opeSub.nunique()

In [ ]:
#在某個當下檢查使用者是否為subscriber的身分(使用free trial者也屬於subscriber之範疇)
#並分別印出yes和no之次數
#其中我將yes之次數轉化為實際人數並列印出來
subr = df["e_segment_map.subscriber"]
yes = subr[subr.values == "yes"]
no = subr[subr.values == "no"]
print(len(yes))
print(len(no))
momentSub = df.loc[np.array(yes.index),"f_device_id"]
momentSub.nunique()

In [ ]:
#為了要確實計算出訂閱者總人數，必須將operation調查出為subscriber與在某個moment下調查出為subscriber扣掉交集
#此處列印出交集元素device_id並計算共有幾個這樣的case
count = 0
for ele in opeSub.unique():
    for element in momentSub.unique():
        if element == ele:
            count = count+1
            print(element)
print(count)

In [ ]:
#此處會過濾出overall的subscriber名單名稱為con為一個nparray
count=0
con = opeSub.unique()
#這邊剛好只有一位交集，若有好幾位，可能con = []就要將交集id包成list
con = np.delete(con, np.where(con == "c8b2c217aa895fd5ad39b92bfe5afb1c0c4f2eb0"))
for element in momentSub.unique():
    if element == ele:
        count = count+1
        print(element)
    else:
        con = np.append(con,element)

print(len(con))

In [ ]:
#在df中找出con名單裡的device_id並將其完整資訊存入rslt_df這個dataFrame裡
rslt_df = df[df['f_device_id'].isin(con)]

In [ ]:
#合併timeStamp之Date和Time的column，並將其結果存入period變數
rslt_df['period'] = rslt_df[['e_timestamp_day', 'e_timestamp_time']].apply(lambda x: ' '.join(x), axis=1)
#轉成timeStamp形式
rslt_df["period"]= pd.to_datetime(rslt_df["period"])
#針對個別使用者將其行為按照時間排序
test = rslt_df.sort_values(['f_device_id','period']).groupby('f_device_id')
#將排序過的訂閱者資訊存入csv裡，名稱記得命名為batch的編號
final = test.head(len(rslt_df['period']))
final.to_csv("update.csv")